## Examples of `mystreams` module usage 

This notebook contains various examples, which demonstrate the capabilities of `mystreams` module: streaming toolkit for PySDK.

The script also uses either a web camera or local camera connected to the machine running this code. The camera index or URL needs to be specified in [env.ini](env.ini) file by defining `CAMERA_ID` variable. If not specified, default local camera will be used.

### Simplest example: video source is connected to video display

In [ ]:
from mystreams import *

# create Composition: an object, which keeps all streaming blocks (called gizmos) and runs them in parallel
c = Composition()

# create gizmos:
source = VideoSourceGizmo()  # video source gizmo
display = VideoDisplayGizmo(
    "press `x` or `q` to stop", allow_drop=False
)  # video display gizmo

# Create pipeline: connect display input to source output
display.connect_to(source)

# add gizmos to composition
c.add(source)
c.add(display)

# start composition
c.start()

# call c.stop() to stop composition or just press `x` or `q` in display window


### Same example, but with compact syntax

In [ ]:
from mystreams import *

c = Composition()

# Create gizmos and pipeline as a single-liner:
# we use __call__() operator of Composition class instead of add() method
# and we use `>>` operator of gizmo classes instead of connect_to() method
c(VideoSourceGizmo()) >> c(VideoDisplayGizmo())

c.start()


### Example of forked streams

In [ ]:
from mystreams import *

c = Composition()

# create and add to composition all required gizmos
source = c.add(VideoSourceGizmo())  # video source gizmo
display = c.add(
    VideoDisplayGizmo(["Original", "Resized"])
)  # two-input display gizmo: will show two windows
resizer = c.add(ResizingGizmo(300, 200))  # resizing gizmo

# Create pipeline: the image source is fed to a display and to the image resizing gizmo,
# which is then fed to another display.

display.connect_to(source, 0)  # display input 0 is "Original"
resizer.connect_to(source)
display.connect_to(resizer, 1)  # display input 1 is "Resized"

c.start()


### Example of capturing video input into file with simultaneous display on a screen

In [ ]:
from mystreams import *

c = Composition()

source = c.add(VideoSourceGizmo())
display = c.add(VideoDisplayGizmo())
saver = c.add(VideoSaverGizmo("mycapture.mp4"))

source >> display
source >> saver

c.start()


### Example of AI inference from the camera with AI inference results display

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you just need to uncomment **one** of the lines in the code below.

You also need to specify your cloud API access token, cloud zoo URLs, and AI server hostname in [env.ini](env.ini) file, located in the same directory as this notebook.

In [ ]:
import degirum as dg, mytools

cloud_token = mytools.get_token()  # get cloud API access token from env.ini file
cloud_zoo_url = mytools.get_cloud_zoo_url()  # get cloud zoo URL from env.ini file

#
# Please UNCOMMENT only ONE of the following lines to specify where to run AI inference
#

# 1. Inference on the DeGirum Cloud Platform
zoo = dg.connect_model_zoo("dgcps://cs.degirum.com" + cloud_zoo_url, cloud_token)

# 2. Inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN
# zoo = dg.connect_model_zoo((mytools.get_ai_server_hostname(), "https://cs.degirum.com" + cloud_zoo_url), cloud_token)

# 3. Inference on DeGirum ORCA accelerator installed on your computer
# zoo = dg.connect_model_zoo("https://cs.degirum.com" + cloud_zoo_url, cloud_token)


from mystreams import *

# load some object detection AI model
model = zoo.load_model("yolo_v5s_coco--512x512_quant_n2x_orca_1")

c = Composition()

# create gizmos
source = c.add(VideoSourceGizmo())  # video source
detection = c.add(AiSimpleGizmo(model))  # AI model
display = c.add(
    VideoDisplayGizmo("Detection", show_ai_overlay=True, show_fps=True)
)  # display

# create pipeline
source >> detection >> display

c.start()


### Example of pipelined AI inference

Frames from the camera are supplied to the face detection model and then the mask classification model is applied for each detected face bounding box.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you just need to uncomment **one** of the lines in the code below.

You also need to specify your cloud API access token, cloud zoo URLs, and AI server hostname in [env.ini](env.ini) file, located in the same directory as this notebook.

In [ ]:
import degirum as dg, mytools

cloud_token = mytools.get_token()  # get cloud API access token from env.ini file
cloud_zoo_url = mytools.get_cloud_zoo_url()  # get cloud zoo URL from env.ini file

#
# Please UNCOMMENT only ONE of the following lines to specify where to run AI inference
#

# 1. Inference on the DeGirum Cloud Platform
zoo = dg.connect_model_zoo("dgcps://cs.degirum.com" + cloud_zoo_url, cloud_token)

# 2. Inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN
# zoo = dg.connect_model_zoo((mytools.get_ai_server_hostname(), "https://cs.degirum.com" + cloud_zoo_url), cloud_token)

# 3. Inference on DeGirum ORCA accelerator installed on your computer
# zoo = dg.connect_model_zoo("https://cs.degirum.com" + cloud_zoo_url, cloud_token)


from mystreams import *

# load AI models
face_model = zoo.load_model("yolo_v5s_face_det--512x512_quant_n2x_orca_1")
mask_model = zoo.load_model("mobilenet_v2_mask_yn_cls--224x224_float_n2x_orca_1")
face_model.overlay_show_probabilities = True


class MaskDetectionGizmo(AiGizmoBase):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._cur_result = None

    def on_result(self, result):

        # here result.info contains StreamData object used for AI inference (because AiGizmoBase does it this way);
        # and result.info.meta contains metainfo dictionary placed by AiObjectDetectionCroppingGizmo,
        # because in our pipeline it is connected as a source of this gizmo
        meta = result.info.meta
        if "original_result" in meta:  # new result
            if self._cur_result is not None:
                # send previous result
                self.send_result(StreamData(self._cur_result.image, self._cur_result))
            self._cur_result = meta["original_result"]

        if "cropped_index" in meta:
            # apply mask presence/absence label
            if len(result.results) > 0:
                self._cur_result.results[meta["cropped_index"]][
                    "label"
                ] = result.results[0]["label"]
                self._cur_result.results[meta["cropped_index"]][
                    "score"
                ] = result.results[0]["score"]


c = Composition()

# create gizmos
source = c.add(VideoSourceGizmo())  # video source
face_detection = c.add(
    AiObjectDetectionCroppingGizmo(["face"], face_model)
)  # face detection gizmo, which outputs cropped images for all detected faces
mask_detection = c.add(MaskDetectionGizmo(mask_model))  # mask detection AI model
face_display = c.add(
    VideoDisplayGizmo("Faces", show_ai_overlay=True, show_fps=True)
)  # display

# create pipeline
source >> face_detection >> mask_detection >> face_display

c.start()
